Libraries require and Model

In [1]:

import numpy as np 
import pandas as pd





In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [6]:
df1 = pd.read_csv('/content/drive/MyDrive/fake_job_postings.csv')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [7]:
df1.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [8]:
df1.shape

(17880, 18)

In [9]:
df=df1[['description','requirements','fraudulent']]
df.head()

,description,requirements,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0


In [10]:
###Drop Nan Values
df=df.dropna()

In [11]:
df.shape

(15185, 3)

In [12]:
## Get the Independent Features

X=df.drop('fraudulent',axis=1)

In [13]:
## Get the Dependent features
y=df['fraudulent']

In [14]:
y.value_counts()

0    14473
1      712
Name: fraudulent, dtype: int64

In [15]:
import tensorflow as tf
tf.__version__

'2.9.2'

In [17]:
voc_size=5000

>  ****One-hot Representation****

In [18]:
message = X.copy()

In [19]:
message['description'][1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90

In [20]:
message.reset_index(inplace=True)

In [21]:
import nltk
import re
from nltk.corpus import stopwords

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus[1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom servic rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http url fbe afac cd c f b eef e e f ca dd second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product suc

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[671,
 4031,
 441,
 3796,
 2838,
 4462,
 3099,
 492,
 4334,
 844,
 4301,
 99,
 2786,
 1624,
 1917,
 2484,
 2918,
 1022,
 477,
 2650,
 1523,
 4813,
 1352,
 144,
 3099,
 2964,
 3830,
 3375,
 8,
 321,
 1995,
 2328,
 3234,
 952,
 977,
 144,
 532,
 4462,
 3099,
 4780,
 1963,
 4068,
 1434,
 884,
 2840,
 2786,
 2786,
 1934,
 3655,
 1352,
 2840,
 2786,
 3893,
 4380,
 1269,
 144,
 2550,
 3577,
 3145,
 2202,
 4885,
 3411,
 4969,
 3182,
 441,
 3373,
 107,
 4323,
 4581,
 635,
 4819,
 1169,
 4068,
 1563,
 4947,
 2150,
 793,
 3134,
 1157,
 3193,
 2214,
 3161,
 4581,
 4068,
 2214,
 3147,
 3139,
 952,
 4813,
 1352,
 144,
 3099,
 4935,
 694,
 3492,
 4439,
 311,
 1535,
 2810,
 949,
 3485,
 2277,
 2277,
 2810,
 3253,
 4950,
 3139,
 952,
 4813,
 1352,
 144,
 3099,
 1606,
 2202,
 3773,
 13,
 1022,
 357,
 1693,
 1352,
 1002,
 1349,
 3362,
 357,
 952,
 1213,
 337,
 2786,
 3686,
 4813,
 1046,
 3026,
 3139,
 3632,
 3351,
 3592,
 2896,
 2501,
 161,
 3577,
 2609,
 1352,
 144,
 519,
 2786,
 405,
 1763,
 1352,
 28

****Embedding Representation****

In [26]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[4214 1002 2786 ...  820 2472  835]
 [1934 2420 1116 ... 1046  616  183]
 [   0    0    0 ... 1811 2250 4704]
 ...
 [3377 2632 4407 ... 1428 2143  157]
 [2457 1945 3080 ... 4952 2150 2840]
 [1944 4383 3188 ... 1347 3086 2002]]


In [27]:
embedded_docs[0]

array([4214, 1002, 2786, 1216, 4875, 4529, 1744, 1621, 4704, 2692, 4559,
       4559, 4704,  482,  201, 3849, 4704, 2823, 4704,  828, 2301, 4401,
       1286, 1700, 4500, 1917, 3577,  289, 2288,  532, 4031,  381,  363,
       2668, 4112, 1716, 2425,  820, 2472,  835], dtype=int32)

In [39]:
## Creating model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100))) 
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 50)            250000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape

(15185, (15185,))

In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [31]:
X_final[1]

array([1934, 2420, 1116, 1330, 3622, 3858, 4935, 3139, 3362, 1895, 1352,
       1754, 4969, 2202, 2421,  952,  791, 2421, 1078, 4332,   74, 3740,
       1578, 4500, 4490, 3739, 4225, 2880, 4401,   75, 4490, 4500, 2347,
       2214, 2373, 1477, 1281, 1046,  616,  183], dtype=int32)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [33]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)

Epoch 1/12
178/178 [==============================] - 11s 19ms/step - loss: 0.1765 - accuracy: 0.9593 - val_loss: 0.1490 - val_accuracy: 0.9581
Epoch 2/12
178/178 [==============================] - 2s 10ms/step - loss: 0.0817 - accuracy: 0.9766 - val_loss: 0.1334 - val_accuracy: 0.9605
Epoch 3/12
178/178 [==============================] - 2s 10ms/step - loss: 0.0570 - accuracy: 0.9838 - val_loss: 0.1375 - val_accuracy: 0.9650
Epoch 4/12
178/178 [==============================] - 2s 11ms/step - loss: 0.0398 - accuracy: 0.9886 - val_loss: 0.1430 - val_accuracy: 0.9673
Epoch 5/12
178/178 [==============================] - 2s 10ms/step - loss: 0.0257 - accuracy: 0.9929 - val_loss: 0.1537 - val_accuracy: 0.9658
Epoch 6/12
178/178 [==============================] - 2s 10ms/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 0.2077 - val_accuracy: 0.9579
Epoch 7/12
178/178 [==============================] - 2s 10ms/step - loss: 0.0120 - accuracy: 0.9968 - val_loss: 0.1995 - val_accuracy: 0.960

**Model Performance and Accuracy**

In [34]:
y_pred=(model1.predict(X_test) > 0.5).astype("int32")
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

119/119 [==============================] - 1s 3ms/step


array([[3523,   62],
       [  94,  118]])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9589149328417171

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98      3585
           1       0.66      0.56      0.60       212

    accuracy                           0.96      3797
   macro avg       0.81      0.77      0.79      3797
weighted avg       0.96      0.96      0.96      3797



In [37]:
b=model1.save("model1.h5")
b

In [38]:
input_text=input("job post:")
voc_size=5000
onehot_repr=[one_hot(input_text , voc_size)] 
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
result=(model1.predict(embedded_docs) > 0.5).astype("int32")
result


job post:VisionX works with world-leading brands, Fortune 1000 as their innovation partner, providing product strategy and custom application development leveraging agile methodologies, technology accelerators, and creating Intellectual Property. VisionX has been listed in the Top 10 Most Innovative Companies of 2020 by Fast Company – ranked alongside the likes of Microsoft & Snap Inc. We develop cutting-edge software products integrating computer vision, 3D modeling, AR, VR, decision sciences, and IoT addressing a wide variety of use cases across different industries.
1/1 [==============================] - 0s 23ms/step


array([[0]], dtype=int32)